In [ ]:
import os

import transformers
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("dark_background")
%matplotlib inline

In [ ]:
%cd ..

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
pd.set_option("display.max_colwidth", 150)

In [ ]:
doc_id_t5_path = "output/doc_id_generation_model/best_checkpoint/"

In [ ]:
model = transformers.T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base-multi-sum")#("output/doc_id_generation_model/best_checkpoint/")

In [ ]:
#tokenizer = transformers.AutoTokenizer.from_pretrained("output/doc_id_generation_model/best_checkpoint/")

In [ ]:
#model = model.cuda().half()

In [ ]:
imports_df = pd.read_feather("output/selected_python_files_imports.feather")
files_df = pd.read_feather("output/selected_python_files.feather")

In [ ]:
def get_predicted_path_summary(texts, n_beams=16, max_length=256, max_label_length=64, min_length=32):
    inputs = tokenizer(texts, max_length=max_length,  truncation=True,
                        padding="max_length", return_tensors="pt")
    summaries = model.generate(input_ids=inputs["input_ids"].to(model.device),
                     attention_mask=inputs["attention_mask"].to(model.device),
                     length_penalty=0.8, num_beams=n_beams, max_length=max_label_length, min_length=min_length)
    return tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
import datasets
from functools import partial
import ast
from github_search import seq2seq_utils

In [ ]:
paperswithcode_df = pd.read_csv("output/papers_with_readmes.csv")

In [ ]:
imports_df = pd.read_feather("output/selected_python_files_imports.feather")
files_df = pd.read_feather("output/selected_python_files.feather")

In [ ]:
repo_tasks = paperswithcode_df['tasks'].apply(lambda ts: ", ".join(ast.literal_eval(ts)))
repo_tasks = pd.DataFrame({"repo": paperswithcode_df['repo'], "tasks": repo_tasks})

In [ ]:
files_with_tasks_df = repo_tasks.merge(files_df, on='repo')

In [ ]:
seq2seq_dataset = datasets.load_from_disk("output/seq2seq_hf_dataset/")

In [ ]:
paperswithcode_area_ds= datasets.Dataset.from_pandas(pd.read_csv("data/paperswithcode_tasks.csv").dropna()[['area', 'task_description']])

In [ ]:
paperswithcode_area_ds

In [ ]:
plbart_str = "uclanlp/plbart-single_task-en_python"#uclanlp/plbart-python-en_XX"

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(plbart_str)

In [ ]:
classifier_model = transformers.AutoModelForSequenceClassification.from_pretrained(plbart_str, num_labels=len(set(paperswithcode_area_ds['area'])))

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["task_description"], padding="max_length", truncation=True, max_length=128)

In [ ]:
from sklearn import preprocessing

In [ ]:
import sentence_transformers
cross_encoder = sentence_transformers.cross_encoder.CrossEncoder(plbart_str, max_length=128)
files_sample = files_with_tasks_df.iloc[:5000]

In [ ]:
example_contents = seq2seq_dataset['contents'][::32][:10]
example_doc_idxs = seq2seq_dataset['doc_id'][::32][:10]

In [ ]:
pairs = list(map(tuple, files_with_tasks_df[['tasks', 'repo', 'path', 'content']].itertuples(index=False)))

In [ ]:
from IPython.utils import io
import tqdm

pairs_with_sims = []
for i in tqdm.tqdm(range(1000)):
    print()
    pair = pairs[i]
    file = pair[-1]
    tasks = pair[0]
    full_path = pair[1] + "/" + pair[2]
    sim = cross_encoder.predict([["tasks: " + tasks, file]]).tolist()[0]
    pairs_with_sims.append((*pair, sim))

In [ ]:
pairs_with_sims_df = pd.DataFrame(pairs_with_sims, columns=["tasks", "repo", "path", "contents", "similarity"])
pairs_with_sims_df['similarity'].plot.hist()

In [ ]:
pairs_with_sims_df.groupby("repo").agg(["mean"]).plot.hist()

In [ ]:
pairs_with_sims_df.groupby("repo").agg(["max"]).plot.hist()

In [ ]:

__, tokenizer = seq2seq_utils.get_seq2seq_model_with_tokenizer("Salesforce/codet5-base-multi-sum")

In [ ]:
#tokenizer = transformers.RobertaTokenizerFast.from_pretrained("Salesforce/codet5-base-multi-sum") 

In [ ]:
decoder_start_token_id = tokenizer("<PATH_TASK_SEP>", add_special_tokens=False)['input_ids'][0]

In [ ]:
max_length = 64
inputs = tokenizer(example_contents, max_length=max_length,  truncation=True,
                        padding="max_length", return_tensors="pt")

In [ ]:
outputs = model.generate(input_ids=inputs["input_ids"].to(model.device),
             attention_mask=inputs["attention_mask"].to(model.device),
             length_penalty=0.8, num_beams=10, max_length=64, min_length=8)#, decoder_start_token_id=decoder_start_token_id)

In [ ]:
tokenizer.decode(outputs[7].tolist())

In [ ]:
output_paths = tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
predicted_doc_ids_df = pd.DataFrame.from_records([p.split("<PATH_TASK_SEP>") for p in output_paths], columns=["path", "tasks"])

In [ ]:
example_doc_ids_df = pd.DataFrame.from_records([p.split("<PATH_TASK_SEP>") for p in example_doc_idxs], columns=["path", "tasks"]) 

In [ ]:
predicted_doc_ids_df['tasks']

In [ ]:
example_doc_ids_df['tasks']

In [ ]:
model_inputs = [content + " predict docID: " for (content, doc_id) in zip(example_contents, example_doc_idxs)]

In [ ]:
model_inputs[1]

In [ ]:
sru_contents = '''

import torch
from torch import nn
from typing import List
import os
import json
import sru


rnn_class_type_mapping = {"lstm": nn.LSTM, "sru": sru.SRU}


class SentenceRNN(nn.Module):
    """
    sentence_transformers RNN wrapper
    """
'''

In [ ]:
dimension_reduction_contents = '''

import numpy as np
import tqdm
from sklearn import decomposition


class IncrementalHyperbolicMDS:
    def __init__(self, n_components, dtype="float16"):
        self.ipca = decomposition.IncrementalPCA(n_components=n_components)
        self.dtype = dtype

    def partial_fit(self, D):
        Y = -np.cosh(D)
        self.ipca.partial_fit(Y)
'''

In [ ]:
recommender_contents = '''

import scipy
import pandas as pd
import numpy as np


from sklearn import compose, feature_extraction, metrics
from functools import reduce, partial
import attr
from typing import Union
import umap
import altair


from game_recommender import steam_data
'''# + " predict docID: lambdaofgod <REPO_NAME_SEP> mlutil <REPO_PATH_SEP> mlutil/recommendation.py"

In [ ]:
text_mining_content = '''
def get_wordnet_similarity(
    word, another_word, similarity_method="resnik", pos=None, ic=None
):
    if ic is None:
        ic = wordnet_ic.ic("ic-semcor.dat")
    assert similarity_method in [
        "lin",
        "jcn",
        "resnik",
    ], "Unsupported similarity method: " + str(similarity_method)
    word_synset = wn.synsets(word, pos)[0]
    another_word_synset = wn.synsets(another_word, pos)[0]
    if similarity_method == "lin":
        return word_synset.lin_similarity(another_word_synset, ic)
    elif similarity_method == "jcn":
        return word_synset.jcn_similarity(another_word_synset, ic)
    else:
        return word_synset.res_similarity(another_word_synset, ic)
'''

In [ ]:
zsl_content = '''
import numpy as np
import attr
from toolz import partial
from scarce_learn.zero_shot import zsl_base

from sklearn import preprocessing
import torch
from torch import nn, optim
from scarce_learn.zero_shot import torch_util


class DEVISELayer(nn.Module):

    def __init__(self, n_features, n_class_features, margin, init_weights_std=0.1):
        super(DEVISELayer, self).__init__()
        init_weights = init_weights_std * torch.randn(n_features, n_class_features) 
        self.weights = nn.Parameter(data=init_weights.cuda())
        self.margin = margin

    def forward(self, X, y, label_embeddings):
        loss = torch.Tensor([0]).cuda()
        for i in range(X.shape[0]):
            loss += self._devise_loss(X[i], y[i], label_embeddings)
        return loss

    def _devise_loss(self, embedding, label, label_embeddings):
        indicator = torch.ones(label_embeddings.shape[0], dtype=bool)
        indicator[label] = 0
        per_class_loss = torch_util.similarity_based_hinge_loss(self.weights, embedding, label, label_embeddings)
        return nn.ReLU()(self.margin + per_class_loss).sum()

    def predict(self, X, label_embeddings):
        class_similarities = torch_util.bilinear_feature_similarity(self.weights, X, label_embeddings)
        return torch.argmax(class_similarities, axis=1)

'''

In [ ]:
evolutionary_content = '''
import attr
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter
import pandas as pd
import tqdm
import logging


try:
    import numba
except ImportError as e:
    logging.warning(
        "numba not found, you'll not be able to use mlutil.evolutionary_algorithms.multiobjective"
    )


def bounded_gaussian_noise_mutation(x, n_mutants, lo=0, hi=1, sigma=1e-2):
    noise = sigma * np.random.randn(n_mutants, x.shape[-1])
    return np.clip(x + noise, lo, hi)


@attr.s
class NSGAII:

    optimized_function = attr.ib()
    chromosome_size: int = attr.ib()
    mutation_function = attr.ib(default=bounded_gaussian_noise_mutation)
    random_initializer = attr.ib(default=np.random.rand)
    population_bounds = attr.ib(default=(0, 1))
    objective_names = attr.ib(default=("1st objective", "2nd objective"))
'''

In [ ]:
optimal_transport_content = '''
import numpy as np
import ot
import pandas as pd
import seaborn as sns
from sklearn.metrics.pairwise import cosine_distances


def get_stem_vectors(filtered_stems, keyed_vectors):
    return np.vstack(
        [
            np.mean([keyed_vectors[w] for w in stem_list], axis=0)
            for stem_list in filtered_stems
            if len(stem_list) > 0
        ]
    )


def get_word_vector_optimal_transport(
    word_vectors1, word_vectors2, ot_method=ot.sinkhorn, reg=0.01, normalize_dists=True
):
    cost = cosine_distances(word_vectors1, word_vectors2)
    height, width = cost.shape
    a = np.ones(height)
    b = np.ones(width)
    if normalize_dists:
        a = a / a.sum()
        b = b / b.sum()
    ot_matrix = ot_method(a, b, cost, reg=reg)
    return ot_matrix, (ot_matrix * cost).sum()

'''

In [ ]:
#model = model.cuda().half()

In [ ]:
optimal_transport_content

In [ ]:
model.device

In [ ]:
path_content = " predict docID: lambdaofgod <REPO_NAME_SEP> mlutil <REPO_PATH_SEP> mlutil/recommendation.py"

In [ ]:
tokenizer.decode(
    model.generate(**(tokenizer(path_content, return_tensors="pt").to(model.device)),
    num_beams=5, max_length=128, min_length=32)[0].tolist(), top_k=0, top_p=0.9
)

In [ ]:
paths = ["_nbdev.py", "haystack_search.py", "rss_feeds.py", "zero_shot_learning.py"]

In [ ]:
paths_series = "lambdaofgod <REPO_NAME_SEP> pytorch_hackathon <REPO_PATH_SEP> pytorch_hackathon <PATH_TASK_SEP> " + pd.Series(paths)

In [ ]:
tokenizer.decode(
    model.generate(**(tokenizer(paths_series[3], return_tensors="pt").to(model.device)),
    num_beams=10, max_length=128, min_length=16)[0].tolist(), #, top_p=0.8
)

In [ ]:
example = seq2seq_dataset[:32]

In [ ]:
import torch

In [ ]:
def mean_pooling(model_output, attention_mask):
    # Extract the token embeddings
    token_embeddings = model_output[0]
    # Compute the attention mask
    input_mask_expanded = (attention_mask
                           .unsqueeze(-1)
                           .expand(token_embeddings.size())
                           .float())
    # Sum the embeddings, but ignore masked tokens
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    # Return the average as a single vector
    return sum_embeddings / sum_mask

In [ ]:
model = model.cuda()

In [ ]:
def embed_text(examples, tokenize=False):
    if tokenize:
        inputs = tokenizer(examples["text"], padding=True, truncation=True,
                       max_length=128, return_tensors="pt")
    else:
        inputs = {"input_ids": torch.tensor(examples["input_ids"]).to(model.device), "attention_mask": torch.tensor(examples["attention_mask"]).to(model.device)}
    with torch.no_grad():
        model_output = model.encoder(**inputs)
    pooled_embeds = mean_pooling(model_output, inputs["attention_mask"])
    return {"embedding": pooled_embeds.cpu().numpy()}

In [ ]:
embs_dataset = seq2seq_dataset.train_test_split(test_size=10000)['test'].map(embed_text,
    batched=True,
    batch_size=128
)

In [ ]:
embs_dataset[0].keys()#['labels'][0]

In [ ]:
embs_dataset.add_faiss_index("embedding")

In [ ]:
tokenizer.tokenize("metric learning", return_tensors="pt")

In [ ]:
type(embs_dataset[0]['embedding'])

In [ ]:
query_text = "implements zero-shot learning"

In [ ]:
query_emb = model.encoder(**tokenizer(query_text, return_tensors="pt").to(model.device))

In [ ]:
query_emb = query_emb.last_hidden_state.mean(axis=1)[0].to("cpu").detach().numpy()

In [ ]:
import numpy as np

In [ ]:
#query_emb = np.array(embs_dataset[0]['embedding'])

In [ ]:
files_df.iloc[0]['content']

In [ ]:
files_sample = files_with_tasks_df.iloc[::500]

In [ ]:
files_sample.head()

In [ ]:
[embs_dataset["contents"][i] for i in list(embs_dataset.search("embedding", query=query_emb.astype("float32")).indices)]

In [ ]:
embs_dataset

In [ ]:
import torch

In [ ]:
example_imports

In [ ]:
model.generate(**(tokenizer(example_imports, return_tensors="pt").to(model.device)),
    num_beams=20, max_length=128, min_length=32)

In [ ]:
encoder_output = model.encoder(**inputs.to(model.device)) #decoder_input_ids=torch.tensor([[decoder_start_token_id]]).to(model.device))

In [ ]:
unique_tasks = files_with_tasks_df['tasks'].str.split(',').explode().str.strip().unique()

In [ ]:
unique_tasks 

In [ ]:
encoder_output.last_hidden_state.shape

In [ ]:
tokenizer.decode((-lm_output.logits).argsort()[:,:,:5][0,0].tolist())

# TODO wyławianie istotnych informacji z konfigów

In [ ]:
%%time
generated_doc_ids = get_predicted_path_summary(example_contents, max_length=64, min_length=16, max_label_length=32)

In [ ]:
predicted_tasks = [d for d in generated_doc_ids]

In [ ]:
true_tasks = [d for d in example_doc_idxs]

In [ ]:
for p in zip(predicted_tasks, true_tasks, example_contents):
    print("#")
    print(p[2])
    print(p[0])
    print(p[1])